# Install

In [ ]:
!pip install -q neo4j-driver
!pip install -q gradio

# Imports

In [ ]:
from neo4j import GraphDatabase
import spacy
import gradio as gr

In [ ]:
driver = GraphDatabase.driver("bolt://54.234.162.150:7687", 
                              auth=("neo4j", 
                                    "code-holder-sled"))

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Functions

In [ ]:
def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None

In [ ]:
def chatbot(input, history=[]):
    output = get_answer(input)
    history.append((input, output))
    return history, history

In [ ]:
def get_answer(question):
    doc = nlp(question)
    movie_title = extract_entity(question, ["WORK_OF_ART", "MOVIE"])
    director_name = extract_entity(question, ["PERSON"])
    if movie_title is None:
        return "Sorry, I don't understand what movie you're asking about."
    
    with driver.session() as session:
        if "release" in question or "released" in question:
            result = session.run(f"MATCH (m:Movie) WHERE m.title = '{movie_title}' RETURN m.released")
            if result.peek() is None:
                return f"No information found for the movie '{movie_title}'"
            for record in result:
                return f"The release date of '{movie_title}' is: {record['m.released']}"
        else:
            return "Sorry, I don't understand what you're asking."

# Interface

In [ ]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)